In [19]:
import requests
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
from pandas import DataFrame

def to_int(x):
    if type(x) is int:
        return x
    return int(x.replace(",","").replace(".",""))

In [20]:
url = "https://www.careerlink.vn/vieclam/list?keyword_use=A"
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')

total_jobs_r = soup.find("div", class_="col-md-7").get_text()

total_jobs = total_jobs_r.split()[2]
total_jobs = to_int(total_jobs)


In [21]:
url = "https://www.careerlink.vn/tim-viec-lam-nhanh"

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')
soup.prettify()

full_job_inds_dict = {}
full_ind = []

job_inds = soup.find('div', class_='masonry-container').find_all('li')

for ind in job_inds:
    job_ind = ind.find('a').get_text()
    full_ind.append(job_ind)
    job_ind_no = ind.find('span').get_text()
    job_ind_no = to_int(job_ind_no[1:-1])
    full_job_inds_dict[f'{job_ind}'] = job_ind_no

df1 = pd.DataFrame(full_job_inds_dict.items(), columns = ['Industry','No. of Jobs'])

careerlink_result_ind = df1


In [22]:
#cut last 5 rows of the dataframe and transpose them
#create new dataframe
#merge

url_list = ['https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=I',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=N',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=L',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=T',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=P',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=M',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=D',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=S',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=E'
           ]

senority_list = ['Intern', 'Mới đi làm', 'Nhân viên', 'Kỹ thuật viên', 'Trưởng nhóm', 'Quản lý', 'Giám đốc', 'Quản lí cấp cao', 'Điều hành cấp cao']
senority_no = []
senority_dict = {}

for url in url_list:
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    senority_no_r = soup.find("div", class_="col-md-7").get_text()
    senority_no.append(to_int(senority_no_r.split()[2]))
    
senority_dict = dict(zip(senority_list, senority_no))
senority_dict
    


{'Intern': 160,
 'Mới đi làm': 437,
 'Nhân viên': 15058,
 'Kỹ thuật viên': 869,
 'Trưởng nhóm': 1419,
 'Quản lý': 1567,
 'Giám đốc': 175,
 'Quản lí cấp cao': 13,
 'Điều hành cấp cao': 5}

In [23]:
Careerlink = {'Total Jobs': total_jobs,
              'Industry': full_ind}

Careerlink.update(senority_dict)

df = DataFrame(Careerlink, columns=['Total Jobs', 'Intern', 'Mới đi làm', 'Nhân viên', 'Kỹ thuật viên', 
                                    'Trưởng nhóm', 'Quản lý', 'Giám đốc', 'Quản lí cấp cao', 'Điều hành cấp cao', 
                                    'Industry'])

Careerlink_result = df.merge(df1, on='Industry')

In [24]:
sum_column_1 = Careerlink_result["Mới đi làm"] + Careerlink_result["Nhân viên"] + Careerlink_result["Kỹ thuật viên"]
sum_column_2 = Careerlink_result["Trưởng nhóm"] + Careerlink_result["Quản lý"]
sum_column_3 = Careerlink_result["Giám đốc"] + Careerlink_result["Quản lí cấp cao"] + Careerlink_result["Điều hành cấp cao"]

Careerlink_result["Junior"] = sum_column_1
Careerlink_result["Senior"] = sum_column_2
Careerlink_result['Manager/Director'] = sum_column_3


In [25]:
Careerlink_result = Careerlink_result.drop(['Mới đi làm','Nhân viên','Kỹ thuật viên','Trưởng nhóm','Quản lý',
                                            'Giám đốc','Quản lí cấp cao', 'Điều hành cấp cao',"Industry","No. of Jobs" ],axis=1)

Careerlink_result = Careerlink_result.reindex(columns=['Total Jobs','Intern','Junior','Senior',
                                                       'Manager/Director'])
careerlink_result_sen = Careerlink_result

In [26]:
careerlink_result_sen

,Total Jobs,Intern,Junior,Senior,Manager/Director
0,19703,160,16364,2986,193
1,19703,160,16364,2986,193
2,19703,160,16364,2986,193
3,19703,160,16364,2986,193
4,19703,160,16364,2986,193
5,19703,160,16364,2986,193
6,19703,160,16364,2986,193
7,19703,160,16364,2986,193
8,19703,160,16364,2986,193
9,19703,160,16364,2986,193


In [27]:
careerlink_result_ind

,Industry,No. of Jobs
0,Biên phiên dịch / Thông dịch viên,1037
1,Biên phiên dịch (tiếng Nhật),617
2,Nhân sự,874
3,Pháp lý / Luật,195
4,Thư ký / Hành chánh,1888
5,An Ninh / Bảo Vệ,128
6,Bán lẻ / Bán sỉ,1203
7,Chăm sóc sức khỏe / Y tế,363
8,Dịch vụ khách hàng,2591
9,Phi chính phủ / Phi lợi nhuận,2


In [30]:
from datetime import date
careerlink_result_ind.to_csv(f'careerlink-ind-{date.today()}.csv')
careerlink_result_sen.to_csv(f'careerlink-sen-{date.today()}.csv')